In [1]:
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Trainer, TrainingArguments
import json
import pandas as pd
from datasets import load_dataset, load_from_disk

In [2]:
with open('imp_term.txt', 'r') as file:
    lines = [line.strip() for line in file.readlines()]

In [3]:
ds = load_dataset("Kyudan/MathBridge")

In [4]:
ds_train = ds["train"]

In [5]:
ds_train = ds_train.filter(lambda x: len(x["equation"])>10 and any(word in x['equation'] for word in lines))
ds_train

Filter:   0%|          | 0/23195831 [00:00<?, ? examples/s]

Dataset({
    features: ['context_before', 'equation', 'context_after', 'spoken_English'],
    num_rows: 5942243
})

In [6]:
ds_train_1000 = ds_train.shuffle(seed=42).select(range(10**3))
ds_train_1000

Dataset({
    features: ['context_before', 'equation', 'context_after', 'spoken_English'],
    num_rows: 1000
})

In [7]:
from transformers import pipeline

pipe2 = pipeline("text2text-generation", model="Hyeonsieun/MathSpeech_T5_base_translator", max_length=1000)

In [8]:
import openai

def get_openai_response(prompt, transcription):
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": """You are a helpful assistant. Your task is to correct the input LaTeX code to make it valid and compilable.
                            The input may contain both mathematical and non-mathematical text. Please ensure the output is corrected for both types
                            and that all elements are formatted correctly in LaTeX. Return only the corrected LaTeX code and nothing else.
                            Do not include any extra commands such as documentclass, begin, or end document. Exclude all additional comments, 
                            explanations, and any other text. The original transcription is: """
                + transcription,
            },
            {
                "role": "user",
                "content": prompt,
            },
        ],
    )
    return response.choices[0].message.content

In [9]:
from TeXBLEU.new_metric import texbleu

scores = []
for row in tqdm(ds_train_1000):
    transcription = row["spoken_English"]
    prompt = pipe2(transcription)[0]["generated_text"]
    response = get_openai_response(prompt, transcription)
    score = texbleu(response, row["equation"])
    scores.append(score)
    print(score)

d:\03 My Study Area\5th Semester\IE-643\Project\TeXBLEU\new_metric.py:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  new_embeddings_state = torch.load('new_embeddings.pth

  0%|          | 1/1000 [00:05<1:26:13,  5.18s/it]

Decoded tokens: [' \\', '[', ' \\', 'math', 'cal', '{', 'C', '}', '_', 'K', ' =', ' \\', '{', '1', ',', ' 2', ',', ' \\', 'ld', 'ots', ',', ' K', ' \\', '}', ' \\', ']']
Decoded tokens: ['$', ' C', '_', 'K', ' :', ' =', ' \\', ' {', ' 1', ' ,', ' 2', ' ,', ' ...', ' ,', ' K', ' \\', ' }', ' $']
0.7484


  0%|          | 2/1000 [00:06<47:50,  2.88s/it]  

Decoded tokens: ['$', 'v', '_', 'x', ' =', ' 0', '.', '3', '$']
Decoded tokens: ['$', ' v', '_', 'x', ' =', ' 0', ' $']
0.9572


...

Decoded tokens: ['$', 'T', '^{', '-', '1', '}', '(', 't', ')', '$']
Decoded tokens: ['$', ' \\', 'math', 'cal', ' {', ' T', ' }', ' ^', ' {', ' -', '1', ' }', ' (', ' t', ' )', ' $']
0.7317


100%|█████████▉| 999/1000 [32:30<00:01,  1.49s/it]

Decoded tokens: ['$', ' \\', 'The', 'ta', ' =', ' 0', '.', '5', '$']
Decoded tokens: ['$', ' \\', 'the', 'ta', '=', '0', '.', '5', ' $']
0.977


100%|██████████| 1000/1000 [32:35<00:00,  1.96s/it]

Decoded tokens: ['$', 'A', '_', ' \\', 'mu', ' =', ' [', ' \\', 'math', 'bf', '{', 'A', '}', '_', '1', ' ,', ' \\', 'math', 'bf', '{', 'A', '}', '_', '2', ' ]', '$']
Decoded tokens: ['$', ' A', '_', ' \\', 'mu', ' :', ' =', ' (', ' \\', 'vec', ' {', ' A', ' }', ' _', ' {', ' (', ' 1', ' )', ' }', ' ,', ' \\', ' ,', ' \\', 'vec', ' {', ' A', ' }', ' _', ' {', ' (', ' 2', ' )', ' }', ' )', ' $']
0.8254


In [11]:
final_texbleu = sum(scores) / len(scores)
final_texbleu

0.8435523000000008